# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

Цель исследования - провести исследовательский анализ данных: 
- В избранном регионе найти месторождения, для каждого определить значения признаков;  
- Построить модель и оценить объём запасов;  
- Выбрать месторождения с самыми высокими оценками значений (при этом количество месторождений будет зависить от бюджета компании и стоимости разработки одной скважины);  
- Посчитать прибыль (которая будет равна суммарной прибыли отобранных месторождений). 

Ход исследования. Данные о представлены в 3-х файлах /datasets/geo_data_0.csv, /datasets/geo_data_1.csv, /datasets/geo_data_2.csv, которые содрежат данные геологоразведки трёх регионов, а именно:  
id — уникальный идентификатор скважины;  
f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);  
product — объём запасов в скважине (тыс. баррелей).

О качестве данных ничего неизвестно, поэтому перед проведением исследовательского анализа данных понадобится обхор данных. Данные будут оценены на ошибки и будет оценено их влияние на исследование. Затем, на этапе предобработки данных будет проведен поиск возможности исправления самых критичных ошибок данных.

Исследование пройдет в 4 этапа:

1) Загрузка и подготовка данных.  
2) Обучение и проверка модели.  
3) Подготовка к расчету прибыли.  
4) Расчет прибыли и рисков.  


## Загрузка и подготовка данных

In [3]:
#загрузкa библиотек
import pandas as pd 
import numpy as np
from scipy import stats as st
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [4]:
df_0 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_0.csv')#загрузка файла
df_0.head()#выводим первые строки таблицы

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [5]:
df_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [6]:
df_0.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


In [7]:
df_0.duplicated().sum()

0

In [8]:
df_1 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_1.csv')#загрузка файла
df_1.head(10).sort_values(by = 'product', ascending=False)#выводим первые строки таблицы

,id,f0,f1,f2,product
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
9,jG6Gi,1.069227,-11.025667,4.997844,137.945408
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
4,AHL4O,12.702195,-8.147433,5.004363,134.766305
8,YiRkx,13.355129,-0.332068,4.998647,134.766305
6,h5Ujo,-11.142655,-10.133399,4.002382,110.992147
5,HHckp,-3.327590,-2.205276,3.003647,84.038886
7,muH9x,4.234715,-0.001354,2.004588,53.906522
1,62mP7,14.272088,-3.475083,0.999183,26.953261
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103


In [9]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [10]:
df_1.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


In [11]:
df_1.duplicated().sum()

0

In [12]:
df_2 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_2.csv')#загрузка файла
df_2.head()#выводим первые строки таблицы

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [13]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [14]:
df_2.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


In [15]:
df_2.duplicated().sum()

0

***Вывод***  
По результатам анализа данных установлено отсутствие пропущенных данных и дубликатов данных. 

## Обучение и проверка модели


Обучим и проверим модель для каждого региона.


In [16]:
def check_models(df, name):
    features = df.drop(['product', 'id'], axis=1)
    target = df['product']

    # отделим 25% данных для валидационной выборки
    features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12345)
    
    # инициализируем модель LinearRegression
    model = LinearRegression()
    # обучим модель на тренировочной выборке
    model.fit(features_train, target_train) 
    # получим предсказания модели на валидационной выборке
    predictions_valid = model.predict(features_valid)
    # посчитаем значение метрики RMSE на валидационной выборке
    rmse = (mean_squared_error(target_valid, predictions_valid)**0.5)
    average_amount = predictions_valid.mean()
    #Построим таблицу с основными показателями
    table_lr = pd.DataFrame({'Region_name':[name],
                          'RMSE модели линейной регрессии на валидационной выборке:':[rmse],
                          'Средний запас предсказанного сырья:':[average_amount]})
    predictions_valid = pd.Series(predictions_valid)
    
    return (predictions_valid, target_valid.reset_index()['product'], table_lr)

In [17]:
#Сохраним предсказания, целевой признак и таблицу с метриками
preds_0, target_0, table_df_0 = check_models(df_0, 'Region_0')
preds_1, target_1, table_df_1 = check_models(df_1, 'Region_1')
preds_2, target_2, table_df_2 = check_models(df_2, 'Region_2')
#Выведем общую таблицу с метриками по каждому региону
table = pd.concat([table_df_0, table_df_1, table_df_2])
display(table)

,Region_name,RMSE модели линейной регрессии на валидационной выборке:,Средний запас предсказанного сырья:
0,Region_0,37.579422,92.592568
0,Region_1,0.893099,68.728547
0,Region_2,40.029709,94.965046


***Вывод***  

Исходя из данных, приведенных в таблице выше, наименьшую ошибку
модель показала по региону Region_1.

## Подготовка к расчёту прибыли

При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
Бюджет на разработку скважин в регионе — 10 млрд рублей.
При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.

Все ключевые значения для расчётов сохраним в отдельных переменных.

In [18]:
revenue_per_barrel = 450*10**3#Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей
quantity = 200#200 лучших точек для разработки
costs_per_region= 10**10#Бюджет на разработку скважин в регионе — 10 млрд рублей
best_wells = 200
costs_per_well = costs_per_region/best_wells#Планируемые затраты на одну скважину
print ("Планируемые затраты на одну скважину: {0:.0f} рублей". format (costs_per_well))

Планируемые затраты на одну скважину: 50000000 рублей


Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины. Точка безубыточности означает равенство выручки от реализации фактическим затратам компании. Таким образом, точка безубыточности на одну скважину рассчитывается как:  
Планируемые затраты на одну скважину = Доход с каждой единицы (1 тыс. баррелей) продукта* Количество продукта (в тыс. баррелей).

In [19]:
product = costs_per_well/revenue_per_barrel#Рассчитаем минимальное количество нефти для безубыточной разработки новой скважины
product_per_region = costs_per_region/revenue_per_barrel/1000
print('Минимальное количество нефти для безубыточной разработки новой скважины: {0:.0f} тыс. баррелей'. format (product))
print('Минимальное количество нефти для безубыточной разработки 200 скважин в регионе: {0:.0f} млн баррелей'. format (product_per_region))


Минимальное количество нефти для безубыточной разработки новой скважины: 111 тыс. баррелей
Минимальное количество нефти для безубыточной разработки 200 скважин в регионе: 22 млн баррелей


Сравним полученный объём сырья со средним запасом в каждом регионе.

In [20]:
table = table[['Region_name', 'Средний запас предсказанного сырья:']]
table['breakeven_point'] = table['Средний запас предсказанного сырья:'] - product
table

,Region_name,Средний запас предсказанного сырья:,breakeven_point
0,Region_0,92.592568,-18.518543
0,Region_1,68.728547,-42.382564
0,Region_2,94.965046,-16.146065


Как видно из данных, приведенных в таблице выше, средний запас предсказанного сырья
в каждом из трех регионов ниже точки безубыточности новой скважины.

***Вывод***  
В ходе проведения исследования установлено, для безубыточной разработки скважины минимальное количество нефти в новой скважине должно составлять 111 тыс. баррелей, а для региона - 22 млн баррелей.
При этом средний запас предсказанного сырья в скважинах ниже точки безубыточности во всех регионах.


## Расчёт прибыли и рисков 

Напишем функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели.


In [21]:
#Напишем функцию для расчета прибыли по выбранным скважинам
def profit(target, probabilities, count):
    probs_sorted = probabilities.sort_values(ascending=False)
    selected = target[probs_sorted.index][:count]
    revenue_per_region = selected.sum() * revenue_per_barrel
    return revenue_per_region-costs_per_region

In [22]:
# Напишем функцию для расчета средней прибыли, 95%-й доверительный интервал
def revenue(target, probabilities, name):
    state = np.random.RandomState(12345)
    values = []
    #Применим технику Bootstrap с 1000 выборок для того, чтобы найти распределение прибыли
    for i in range(1000):
        target_subsample = target.sample(500,
                                         replace=True,
                                         random_state=state)
        probs_subsample = probabilities[target_subsample.index]
        values.append(profit(target_subsample, probs_subsample, 200))
        
    values = pd.Series(values)
    lower = values.quantile(0.025)# 
    upper = values.quantile(0.975)
    risk = st.percentileofscore(values, 0)
    mean_values = int(sum(values)/len(values))
   
    table = pd.DataFrame({
                          'Прибыль' 
                          :[values.mean()],
                          '2.5%-квантиль' 
                          :[lower],
                          '97,5%-квантиль'
                          :[upper],
                          'Название'
                          :[name],
                          'Процент риска'
                          :[risk],
                          'Средняя'
                          :[mean_values]
        
                          })
    return table
   
table_0 = revenue(target_0, preds_0, 'first')
table_1 = revenue(target_1, preds_1, 'second')
table_2 = revenue(target_2, preds_2, 'third')
table = pd.concat([table_0, table_1, table_2])
display(table)

,Прибыль,2.5%-квантиль,"97,5%-квантиль",Название,Процент риска,Средняя
0,4.259385e+08,-1.020901e+08,9.479764e+08,first,6.0,425938526
0,5.152228e+08,6.887323e+07,9.315476e+08,second,1.0,515222773
0,4.350084e+08,-1.288805e+08,9.697070e+08,third,6.4,435008362


***Вывод***

Как видно из данных по региону second наименьший риск и составляет 1.0% при этом по этому региону наибольшая средняя. 